In [ ]:
!nvcc --version

In [26]:
%%writefile file1.cu
#include<stdio.h>
#include<cuda.h>

__global__ void dkernel(int *d_mem, int n){
    __shared__ int sharedRow[1024];
    int row= blockIdx.x;
    int col = threadIdx.x;
    int globalIdx = row * n + col;
    sharedRow[col] =d_mem[globalIdx];
    __syncthreads();

    if (col < n - 1) {
        d_mem[globalIdx] += sharedRow[col + 1];
    }
}
int main()
{
  int *h_mem, *d_mem;
  int n=1024;
  h_mem= (int*)malloc(n*n*sizeof(int));

  //intitialize h_mem
  for(int i=0;i<n*n;i++)
  {
      h_mem[i]=i;
  }
  cudaMalloc(&d_mem, n*n*sizeof(int));
  cudaMemcpy(d_mem,h_mem,n*n*sizeof(int),cudaMemcpyHostToDevice);

  //kernels
  dkernel<<<n,n>>>(d_mem, n);


  cudaDeviceSynchronize();
   //copy back to host
  cudaMemcpy(h_mem, d_mem, n*n*sizeof(int), cudaMemcpyDeviceToHost);

  //print result
  printf("Result: okay\n");
  for(int i=0; i< 5; i++)
  {
    for(int j=0; j< 5; j++)
    {
      printf("%d ", h_mem[i*n+j]);
    }
    printf("\n");
  }

  //free memory
  cudaFree(d_mem);
  free(h_mem);
  return 0;
}

Overwriting file1.cu


In [27]:
!nvcc -arch=sm_75 file1.cu -o file1.out

In [28]:
!./file1.out

Result: okay
1 3 5 7 9 
2049 2051 2053 2055 2057 
4097 4099 4101 4103 4105 
6145 6147 6149 6151 6153 
8193 8195 8197 8199 8201 
